In [1]:
import torch
from time import time

def CubicBSpline3D(pts,x,device='cpu',scale=[1.0, 1.0, 1.0],return_jac=False):
    scale=torch.tensor(scale,dtype=torch.float32,device=device)
    
    t_idx=pts.mul(1/scale).floor()
    #t_idx.detach()
    p=torch.arange(pts.size(0))
    #print(p)
    t=pts.mul(1/scale)-t_idx
    #t_idx=torch.tensor(_idx.clone().flatten(),dtype=torch.int32,device=device)
    ones4=torch.ones([4,4,4],dtype=torch.int32,device=device)
    onesp=torch.ones(t.size(0),dtype=torch.int32,device=device)
    t_idx=t_idx.flatten()
    #print(t_idx)
    stride_x, stride_y, stride_z=torch.meshgrid([torch.arange(0,4,device=device)-2, torch.arange(0,4,device=device)-2, torch.arange(0,4,device=device)-2])
    indices1=torch.clamp(torch.einsum('a,bcd->abcd',t_idx[3*p],ones4)+torch.einsum('a,bcd->abcd',onesp,stride_x.int()),0,x.shape[1]-1)*(x.shape[2]*x.shape[3])
    #print(indices.shape)
    indices2=torch.clamp(torch.einsum('a,bcd->abcd',t_idx[3*p+1],ones4)+torch.einsum('a,bcd->abcd',onesp,stride_y.int()),0,x.shape[2]-1)*(x.shape[3])
    #print(indices.shape)
    indices3=torch.clamp(torch.einsum('a,bcd->abcd',t_idx[3*p+2],ones4)+torch.einsum('a,bcd->abcd',onesp,stride_z.int()),0,x.shape[3]-1)
    #print(indices.shape)
    indices=indices1.long()+indices2.long()+indices3.long()
    indices=indices.long()
    
    jac=None
    a=torch.stack([t.flatten()*0+1, t.flatten(), t.flatten()**2, t.flatten()**3],dim=1) 
    b=torch.tensor(([1, 4, 1, 0],[-3, 0, 3, 0],[3, -6, 3, 0],[-1, 3, -3, 1]),dtype=torch.float32,device=device)/6 
    y=torch.mm(a,b)
    w=torch.sum(torch.einsum('ab,ac,ad->abcd',y[3*p,:],y[3*p+1,:],y[3*p+2,:])*x.flatten(start_dim=1)[:,indices.long()],dim=[2,3,4])
    #print(w)
    if(return_jac):
        da=torch.stack([t.flatten()*0, t.flatten()*0+1, t.flatten()*2, 3*t.flatten()**2],dim=1) 
        dy=torch.mm(da,b)
        wx=torch.sum(torch.einsum('ab,ac,ad->abcd',dy[3*p,:],y[3*p+1,:],y[3*p+2,:])*x.flatten(start_dim=1)[:,indices],dim=[2,3,4])
        wy=torch.sum(torch.einsum('ab,ac,ad->abcd',y[3*p,:],dy[3*p+1,:],y[3*p+2,:])*x.flatten(start_dim=1)[:,indices],dim=[2,3,4])
        wz=torch.sum(torch.einsum('ab,ac,ad->abcd',y[3*p,:],y[3*p+1,:],dy[3*p+2,:])*x.flatten(start_dim=1)[:,indices],dim=[2,3,4])
        jac=torch.stack([wx.t()/scale[0],wy.t()/scale[0],wz.t()/scale[0]],dim=2)
    return w,jac
 

def NMI(x,device):
    h1,h2,h3=Histogram2D(x,device)
    p1=h1/h1.sum()
    p2=h2/h2.sum()
    p3=h3/h3.sum()
    HIJ=-torch.sum(p1*torch.log(p1))
    HI=-torch.sum(p2*torch.log(p2))
    HJ=-torch.sum(p3*torch.log(p3))
    #print(HIJ,HI,HJ,p1.sum(),p2.sum(),p3.sum())
    return(HI+HJ)/HIJ

class affine_registration:
    def __init__(self,img_s,img_t):
        self.img_t=img_t
        self.img_t=img_s
        self.resolution_s=img_s.get_header().get_zooms()
        self.resolution_t=img_t.get_header().get_zooms()
        
def createStencil3D(device='cpu'):  
    stencil3D=torch.tensor([[1,0,0],#basis x,y,z
                         [0,1,0],
                         [-1,0,0],
                         [0,-1,0],
                          [0,0,1],
                         [0,0,-1],
                          [1,1,0],# xy
                         [-1,1,0],
                         [1,-1,0],
                         [-1,-1,0],
                         [1,0,1], #xz
                         [-1,0,1],
                         [1,0,-1],
                         [-1,0,-1],
                         [0,1,1], #yz
                         [0,-1,1],
                         [0,1,-1],
                         [0,-1,-1],
                          [1,1,1], #xyz
                         [-1,1,1],
                         [1,1,-1],
                         [-1,1,-1],
                          [1,-1,1], #xyz
                         [-1,-1,1],
                         [1,-1,-1],
                         [-1,-1,-1],
                         ],dtype=torch.float32,device=device).t()
    q=torch.norm(stencil3D,p=2,dim=0)
    stencil3D.div_(torch.stack([q,q,q],dim=0)).t()
    return stencil3D       
def Histogram2D(vals,device):
    rangeh=torch.ceil(vals.max()-vals.min()).long()+4
    t_idx=vals.floor().long()+2
    p=torch.arange(vals.size(0))
    t=vals.double()+2-t_idx
    #print("rangeh: ",rangeh, '\n')
    ones4=torch.ones([2,2],dtype=torch.int32,device=device)
    onesp=torch.ones(t.size(0),dtype=torch.int32,device=device)
    stride_x, stride_y=torch.meshgrid([torch.arange(0,2,device=device)-1, torch.arange(0,2,device=device)-1])
    t_idx=t_idx.flatten()
    indices=torch.einsum('a,bc->abc',t_idx[2*p],ones4)*(rangeh)
    indices+=torch.einsum('a,bc->abc',onesp,stride_x)*rangeh
    indices+=torch.einsum('a,bc->abc',t_idx[2*p+1],ones4)
    indices+=torch.einsum('a,bc->abc',onesp,stride_y)
    y=torch.stack([1-t.flatten(), t.flatten()],dim=1)
    
    res=(torch.einsum('ab,ac->abc',y[2*p,:],y[2*p+1,:]))
   # print("res: ",res.sum(),res.shape,indices.shape)
    sort_res,nid=torch.sort(indices.flatten())
    nres=res.flatten()[nid.flatten()]
    v,ids=sort_res.flatten().unique_consecutive(return_counts=True)
    val=torch.split(nres.flatten(),ids.tolist());
    hist=torch.zeros(v.size(),device=device,dtype=torch.float32)
    for index, value in enumerate(val): 
        hist[index]=value.sum()
    
    sort_res,nid=torch.sort(v.flatten()%rangeh)
    nres=hist.flatten()[nid.flatten()]
    va,ids=sort_res.flatten().unique_consecutive(return_counts=True)
    vala=torch.split(nres.flatten(),ids.tolist());
    hist_a=torch.zeros(va.size(),device=device,dtype=torch.float32)
    
    
    for index, value in enumerate(vala): 
        hist_a[index]=value.sum()
    sort_res,nid=torch.sort((v.flatten()/rangeh).int())
    nres=hist.flatten()[nid.flatten()]
    vb,ids=sort_res.flatten().unique_consecutive(return_counts=True)
    valb=torch.split(nres.flatten(),ids.tolist());
    hist_b=torch.zeros(vb.size(),device=device,dtype=torch.float32)
    
    
    for index, value in enumerate(valb): 
        hist_b[index]=value.sum()
    return hist[hist>0], hist_a[hist_a>0], hist_b[hist_b>0]


In [9]:
#this will load the data in the requird format for the registration

import numpy as np
device='cpu'
#dipy.io.image is for loading / saving imaging datasets dipy.io.gradients is for loading / saving our bvals and bvecs

from dipy.io.image import load_nifti, save_nifti
from dipy.io.gradients import read_bvals_bvecs
#from dipy.core.gradients import gradient_table
#dipy.reconst is for the reconstruction algorithms which we use to create voxel models from the raw data.



from dipy.data import get_fnames
import os

# load data 1
hardi_fname, hardi_bval_fname, hardi_bvec_fname = get_fnames('stanford_hardi')
#Next, we read the saved dataset. gtab contains a GradientTable object (information about the gradients e.g. b-values and b-vectors).

#the path to the data of the reference
path='D://HCP//HCP//100206//T1w//Diffusion//'
hardi_bval_fname=path+'bvals'
hardi_bvec_fname=path+'bvecs'
hardi_fname=path+'data.nii.gz'
hardi_mask_fname=path+'nodif_brain_mask.nii.gz'

data, affine = load_nifti(hardi_fname)
maskdata, maffine = load_nifti(hardi_mask_fname)
mdata=np.repeat(maskdata.reshape([1,145, 174, 145]),288,axis=0)
bvals, bvecs = read_bvals_bvecs(hardi_bval_fname, hardi_bvec_fname)

#selecting b=1000
res= (0 < np.round(bvals/100)*100) * (np.round(bvals/100)*100 < 1500)
res_m= (10 > np.round(bvals/100)*100) 
bvals=np.round(bvals[res]/100)*100

bvecs=bvecs[res,:]
mdata=mdata[res,:,:,:]

#normalizing with b0
td=data[:,:,:,res_m].mean(3)
data=data[:,:,:,res]/(td[:,:,:,None]+1e-4)

#formatting the data to our specification
data1f=torch.tensor(data).permute([3,0,1,2])
nbvecs1=torch.tensor(bvecs)

#the path to the data of the target
path='D://HCP//HCP//100307//T1w//Diffusion//'
hardi_bval_fname=path+'bvals'
hardi_bvec_fname=path+'bvecs'
hardi_fname=path+'data.nii.gz'
hardi_mask_fname=path+'nodif_brain_mask.nii.gz'

data, affine = load_nifti(hardi_fname)
maskdata, maffine = load_nifti(hardi_mask_fname)
mdata=np.repeat(maskdata.reshape([1,145, 174, 145]),288,axis=0)
bvals, bvecs = read_bvals_bvecs(hardi_bval_fname, hardi_bvec_fname)

#selecting b=1000
res= (0 < np.round(bvals/100)*100) * (np.round(bvals/100)*100 < 1500)
res_m= (10 > np.round(bvals/100)*100) 
bvals=np.round(bvals[res]/100)*100
bvecs=bvecs[res,:]

normalizing with b0
mdata=mdata[res,:,:,:]
td=data[:,:,:,res_m].mean(3)
data=data[:,:,:,res]/(td[:,:,:,None]+1e-4)

#formatting the data to our specification
data2f=torch.tensor(data).permute([3,0,1,2])
nbvecs2=torch.tensor(bvecs)


data.shape (145, 174, 145, 90) (145, 174, 145)
data.shape (145, 174, 145, 90) (145, 174, 145)


In [10]:
from time import time
import torch.optim as optim

#setting the concentration parameter of the watson kernel
kappa = 10

#define the evaluation points
s_x, s_y, s_z=(torch.meshgrid([torch.arange(0,70,device=device), torch.arange(0,90,device=device), torch.arange(0,70,device=device)]))
pts=(torch.stack([s_x.flatten(),s_y.flatten()+5,s_z.flatten()],dim=1).float()+2.25)*3

#define the computational grid
x=torch.rand([3, 60, 60, 60], dtype=torch.float32, requires_grad=True,device=device)

#reduce the pointset with a mask


#evaluate the target image
ref,_=CubicBSpline3D(pts,data1f,device,[1,1,1])

#create DWI evaluation orientations
stencil=createStencil3D(device)

#create identity matrices for jacobian computations
lid = torch.eye(3)
lid = lid.reshape((1, 3, 3))
Id = lid.repeat(pts.shape[0], 1, 1).to(device)

#perform watson interpolation over the stencil for all evaluation points
watson_w=kappa*torch.einsum('abc,eb->ace',(torch.nn.functional.normalize(torch.einsum('abc,cd->abd',Id,stencil),2,dim=1)),nbvecs1.float())**2
#normalizing using a trick to avoid overflow
watson_w=torch.exp(watson_w-(watson_w.max(dim=2))[0][:,:,None])
watson_w=watson_w/(watson_w.sum(dim=2)[:,:,None])
ref=torch.einsum('abc,ca->ab',watson_w,ref)

#initialize optimizer
no_iter=200
lr=1
optimizer = optim.Adam([x], lr=lr)
for i in range(no_iter):
 
    st = time()
    #deformation computation
    dp,jac=CubicBSpline3D(pts,x,device,[10.0, 10.0, 10.0],True)
    tpts=pts+dp.t()
    
    #interpolate image
    mov,_=CubicBSpline3D(tpts,data2f,device,[1,1,1])
    
    #perform watson interpolation over the deformed stencil for all evaluation points
    watson_w=kappa*torch.einsum('abc,eb->ace',(torch.nn.functional.normalize(torch.einsum('abc,cd->abd',jac+Id,stencil),2,dim=1)),nbvecs2.float())**2
    #normalizing using a trick to avoid overflow
    watson_w=torch.exp(watson_w-(watson_w.max(dim=2))[0][:,:,None])
    watson_w=watson_w/(watson_w.sum(dim=2)[:,:,None])
    mov=torch.einsum('abc,ca->ab',watson_w,mov)
    
    #prepare data for similarity computation
    pair=torch.stack([mov.flatten(),ref.flatten()],dim=1)
    
    #compute dis-similarity
    cst=-NMI(pair,device)
    cst.backward()
    optimizer.step()
    
    print(cst,time()-st)
    print(i)
    optimizer.zero_grad()


tensor(-1.0543, grad_fn=<NegBackward>) 17.32465171813965
0


KeyboardInterrupt: 

torch.Size([150000, 3])